In [6]:
import tensorflow as tf
from keras import layers

In [7]:
from veggideas.transfer import load_non_trainable_model
from keras import regularizers
from keras import models
from keras import optimizers
from veggideas.load_data import load_train_data, load_val_data

In [8]:
train_data = load_train_data()
val_data = load_val_data()
model = load_non_trainable_model()

Found 15000 files belonging to 15 classes.
Loading training data
Training data successfully loaded ✅
Found 3000 files belonging to 15 classes.
Loading validation data
Validation data successfully loaded ✅


In [9]:
val_data

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 15), dtype=tf.float32, name=None))>

In [15]:
small_val_data = val_data.take(5)
small_train_data = train_data.take(5)

In [20]:
def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    resize_and_rescale = tf.keras.Sequential([
        layers.Rescaling(1./255)])

    data_augmentation = tf.keras.Sequential([
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.2),
        layers.RandomBrightness(factor=0.2),
        layers.RandomZoom(.5, .2),
        layers.RandomContrast(factor=0.2),
        ])

    base_model = model
    flattening_layer = layers.Flatten()
    dense_layer = layers.Dense(128, activation='relu')
    dense_layer_2 = layers.Dense(64, activation='relu')
    reg_l2 = regularizers.L2(0.01)
    dense_layer_reg = layers.Dense(128, activation='relu', bias_regularizer=reg_l2)
    maxpool_layer = layers.MaxPool2D(pool_size=(2,2))
    prediction_layer = layers.Dense(15, activation='softmax')

    model = models.Sequential([
        resize_and_rescale,
        data_augmentation,
        base_model,
        flattening_layer,
        dense_layer,
        dense_layer_reg,
        dense_layer_2,
        prediction_layer
        ])


    opt = optimizers.Adam(learning_rate=1e-3)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model


model_transfer = add_last_layers(model)


model_transfer.fit(small_train_data, batch_size=32, epochs=10, validation_data=small_val_data)

Epoch 1/10
5/5 [==============================] - 26s 5s/step - loss: 2.8335 - accuracy: 0.1000 - val_loss: 2.9403 - val_accuracy: 0.0750
Epoch 2/10
5/5 [==============================] - 21s 5s/step - loss: 2.7991 - accuracy: 0.1125 - val_loss: 2.7950 - val_accuracy: 0.0750
Epoch 3/10
5/5 [==============================] - 22s 5s/step - loss: 2.7662 - accuracy: 0.0688 - val_loss: 2.9371 - val_accuracy: 0.0688
Epoch 4/10
5/5 [==============================] - 22s 5s/step - loss: 2.7299 - accuracy: 0.0875 - val_loss: 2.9637 - val_accuracy: 0.0500
Epoch 5/10
5/5 [==============================] - 21s 5s/step - loss: 2.7016 - accuracy: 0.1437 - val_loss: 2.8218 - val_accuracy: 0.0938
Epoch 6/10
5/5 [==============================] - 21s 5s/step - loss: 2.6719 - accuracy: 0.1437 - val_loss: 2.6835 - val_accuracy: 0.0938
Epoch 7/10
5/5 [==============================] - 21s 5s/step - loss: 2.6732 - accuracy: 0.0625 - val_loss: 2.7311 - val_accuracy: 0.0875
Epoch 8/10
5/5 [==================

In [ ]:
def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    resize_and_rescale = tf.keras.Sequential([
        layers.Rescaling(1./255)])

    data_augmentation = tf.keras.Sequential([
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.2),
        layers.RandomBrightness(factor=0.2),
        layers.RandomZoom(.5, .2),
        layers.RandomContrast(factor=0.2),
        ])

    base_model = model
    flattening_layer = layers.Flatten()
    dense_layer = layers.Dense(128, activation='relu')
    dense_layer_2 = layers.Dense(64, activation='relu')
    reg_l2 = regularizers.L2(0.01)
    dense_layer_reg = layers.Dense(128, activation='relu', bias_regularizer=reg_l2)
    maxpool_layer = layers.MaxPool2D(pool_size=(2,2))
    prediction_layer = layers.Dense(15, activation='softmax')

    model = models.Sequential([
        resize_and_rescale,
        data_augmentation,
        base_model,
        layers.Conv2D(512, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(256, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        flattening_layer,
        dense_layer,
        dense_layer_reg,
        dense_layer_2,
        prediction_layer
        ])


    opt = optimizers.Adam(learning_rate=1e-3)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model


model_transfer = add_last_layers(model)


model_transfer.fit(small_train_data, batch_size=32, epochs=10, validation_data=small_val_data)